In [1]:
!pip install ag2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.5/593.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.3 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_AI_STUDIO')

In [4]:
import autogen

In [90]:
llm_config={"model": "gpt-4-turbo"}
# llm_config={"model": "gemini/gemini-2.0-pro-exp-02-05"}

# Creating Agents

In [91]:
from autogen import ConversableAgent

In [92]:
onboarding_personal_information_agent = ConversableAgent(
    name="onboarding_personal_information_agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE'
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [93]:
onboarding_topic_preference_agent = ConversableAgent(
    name="onboarding_topic_preference_agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [94]:
customer_engagement_agent = ConversableAgent(
    name="customer_engagement_agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [95]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

# Creating Tasks

In [96]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message":
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message":
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

# Starting the Onboarding Process....

In [98]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
onboarding_personal_information_agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------
Replying as customer_proxy_agent. Provide feedback to onboarding_personal_information_agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: Vanessa
customer_proxy_agent (to onboarding_personal_information_agent):

Vanessa

--------------------------------------------------------------------------------
onboarding_personal_information_agent (to customer_proxy_agent):

Thank you, Vanessa. Could you also provide your location?

--------------------------------------------------------------------------------
Replying as customer_p

# Printing out the Summary

In [99]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{
  "name": "Vanessa",
  "location": "Madrid"
}


Vanessa is interested in reading about top beautiful destination wedding locations in the world.


Vanessa expressed interest in reading about top beautiful destination wedding locations around the world. Recommended destinations included Santorini, Tuscany, Maui, Jaipur, Lake Como, Bora Bora, and Ibiza, detailing each location's unique appeal. Vanessa may be planning or dreaming of a destination wedding.




# Printing out the cost

In [100]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.00303, 'gpt-4-turbo-2024-04-09': {'cost': 0.00303, 'prompt_tokens': 207, 'completion_tokens': 32, 'total_tokens': 239}}, 'usage_excluding_cached_inference': {'total_cost': 0.00303, 'gpt-4-turbo-2024-04-09': {'cost': 0.00303, 'prompt_tokens': 207, 'completion_tokens': 32, 'total_tokens': 239}}}


{'usage_including_cached_inference': {'total_cost': 0.00135, 'gpt-4-turbo-2024-04-09': {'cost': 0.00135, 'prompt_tokens': 84, 'completion_tokens': 17, 'total_tokens': 101}}, 'usage_excluding_cached_inference': {'total_cost': 0.00135, 'gpt-4-turbo-2024-04-09': {'cost': 0.00135, 'prompt_tokens': 84, 'completion_tokens': 17, 'total_tokens': 101}}}


{'usage_including_cached_inference': {'total_cost': 0.02189, 'gpt-4-turbo-2024-04-09': {'cost': 0.02189, 'prompt_tokens': 656, 'completion_tokens': 511, 'total_tokens': 1167}}, 'usage_excluding_cached_inference': {'total_cost': 0.02189, 'gpt-4-turbo-2024-04-09': {'cost': 0.02189, 'prompt_tokens': 65